In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1662


In [2]:
using PProf
using ProfileCanvas
using BenchmarkTools: @btime

import Random
rng = Random.MersenneTwister();

In [3]:
n = 6
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);
#ham = HarmonicOscillator(ω)

l = 6
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = setup_RHF(system, mixer);
#rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);
energy(rhf)

6.221946627007826

In [7]:
ccsd = setup_CCSD(System(rhf))
@time ccsd = compute_ground_state!(ccsd, max_iter = 1000, tol = 1e-10, verbose=0);
energy(ccsd)

In [121]:
samples = 2^20
metro = Importance(equils = 0, samples = samples, step = 0.1);
#metro = Metropolis(equils = 1e4, samples = samples, step = 0.1);
samples

1048576

In [14]:
nn = NeuralNetwork([Dense(2), Sigmoid, Dense(2), Sigmoid, Dense(1), Exp], n=n, rng = rng);
slater = Slater(rhf);
wf = SlaterNN(slater, nn);

In [17]:
optimizer = GradientDescent(lr = 0.01, max_iter = 1, tol = 1e-6);
@time wf, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 8, verbose = true);

E = 11.687 iter = 1/1                                      
No convergence reached, final norm of gradient was 0.00042251744258121676
  0.032913 seconds (2.37 k allocations: 560.781 KiB)


In [130]:
@time result = blocking(wf, ham, metro, nthreads = 8);
result.E

  0.950827 seconds (2.00 k allocations: 88.510 MiB, 1.37% gc time)


11.441262032268376

In [ ]:
6.355 ms (1910 allocations: 1.20 MiB)

In [30]:
2^20

1048576

In [129]:
@profview blocking(wf, ham, metro, nthreads = 8)

ProfileCanvas.ProfileData(Dict{String, ProfileCanvas.ProfileFrame}("3" => ProfileCanvas.ProfileFrame("root", "", "", 0, 0, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "4" => ProfileCanvas.ProfileFrame("root", "", "", 0, 0, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "1" => ProfileCanvas.ProfileFrame("root", "", "", 0, 499, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#1", "threadingconstructs.jl", ".\\threadingconstructs.jl", 30, 422, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#236#threadsfor_fun", "threadingconstructs.jl", ".\\threadingconstructs.jl", 51, 422, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#236#threadsfor_fun#52", "threadingconstructs.jl", ".\\threadingconstructs.jl", 84, 422, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("macro expansion", "steps.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\steps.jl", 16, 415, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("sampled_steps!", "steps.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\steps.jl", 31, 248, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("metro_step!", "metropolis.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\metropolis.jl", 56, 218, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("consider_qf!", "slaterNN.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\Wavefunctions\\slaterNN.jl", 41, 182, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("consider_qf!", "neuralnetwork.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\Wavefunctions\\neuralnetwork.jl", 249, 96, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("model!", "neuralnetwork.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\Wavefunctions\\neuralnetwork.jl", 88, 61, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("layerEval!", "layer.jl", "C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods\\src\\VMC\\Wavefunctions\\Tools\\layer.jl", 50, 25, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("mul!", "matmul.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\matmul.jl", 276, 25, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("mul!", "matmul.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\matmul.jl", 65, 25, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("gemv!", "matmul.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\matmul.jl", 503, 20, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("gemv!", "blas.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\blas.jl", 666, 14, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), ProfileCanvas.ProfileFrame("gemv!", "blas.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\blas.jl", 642, 3, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), ProfileCanvas.ProfileFrame("gemv!", "blas.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\blas.jl", 0, 1, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), ProfileCanvas.ProfileFrame("_chkstride1", "LinearAlgebra.jl", "C:\\buildbot\\worker\\package_win64\\build\\usr\\share\\julia\\stdlib\\v1.8\\LinearAlgebra\\src\\Linea

In [108]:
function vec_outer!(M::Matrix{Float64}, u, vT)
    @inbounds for (i, ui) in enumerate(u)
        for (j, vi) in enumerate(vT)
            M[i, j] = ui * vi
        end
    end
    return M
end

vec_outer! (generic function with 1 method)

In [116]:
la.BLAS.set_num_threads(8)

In [109]:
n = 20
M = zeros(n, n)
u = rand(n)
vT = rand(n)

@btime vec_outer!(M, u, vT);
@btime la.BLAS.gemm!('N', 'T', 1.0, u, vT, 1.0, M);
@btime la.mul!(M, u, transpose(vT));

  291.760 ns (0 allocations: 0 bytes)
  329.333 ns (0 allocations: 0 bytes)
  411.500 ns (1 allocation: 16 bytes)


In [ ]:
1.630 μs (0 allocations: 0 bytes)
  872.727 ns (0 allocations: 0 bytes)
  1.150 μs (1 allocation: 16 bytes)